In [1]:
# Agricultural Yield Forecasting - Data Collection
# ================================================

import sys
sys.path.append('..')  # Add parent directory to path

import pandas as pd
import numpy as np
from pathlib import Path
import config
from src.data.collectors import download_all_data

print("✅ Imports successful")
print(f"📁 Project root: {config.PROJECT_ROOT}")
print(f"📁 Data directory: {config.DATA_DIR}")

✅ Config loaded - Project root: /Users/evanpoulson/Desktop/Agro Project/notebooks/..


ModuleNotFoundError: No module named 'tqdm'

In [ ]:
# Run the complete data collection pipeline
# This will take 15-30 minutes depending on your internet connection

download_all_data()

In [ ]:
# Check what we downloaded

env_canada_dir = config.RAW_DATA_DIR / "weather" / "environment_canada"
nasa_power_dir = config.RAW_DATA_DIR / "weather" / "nasa_power"
yields_dir = config.RAW_DATA_DIR / "yields"

print("Environment Canada files:")
for file in env_canada_dir.glob("*.csv"):
    df = pd.read_csv(file)
    print(f"  ✅ {file.name}: {len(df)} rows")

print("\nNASA POWER files:")
for file in nasa_power_dir.glob("*.csv"):
    df = pd.read_csv(file)
    print(f"  ✅ {file.name}: {len(df)} rows")

print("\nYield baseline files:")
for file in yields_dir.glob("*.csv"):
    df = pd.read_csv(file)
    print(f"  ✅ {file.name}: {len(df)} rows")

In [ ]:
# Preview one of the weather files

red_deer_file = env_canada_dir / "red_deer_weather.csv"
df = pd.read_csv(red_deer_file)

print(f"Red Deer Weather Data Preview:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()